# Taegis Magic
---

## Assets
---

In [ ]:
import pandas as pd
from taegis_magic.pandas.alerts import inflate_raw_events
from taegis_magic.pandas.assets import lookup_assets

In [ ]:
%load_ext taegis_magic.magics

### `lookup_assets` pipe function
---

This pandas pipe function will work when there are the following columns in the DataFrame:
- A column name that contains `host_id`
- A column name that contains `tenant.id` (or `tenant_id`)
- The region/environment the lookups are taking place

#### Asset Data with Taegis Alerts

**Note:** When working with an alert data dataframe, you will need to use the `inflate_raw_events` pipe function in order to obtain the `event_data.host_id` column.

In [ ]:
%%taegis alerts search --assign alerts_df

FROM alert
WHERE severity >= 0.6 AND
sensor_types CONTAINS 'ENDPOINT'
EARLIEST=-1d
| head 100

In [ ]:
inflated_alerts = alerts_df.pipe(inflate_raw_events)

inflated_alerts.head(30)[["metadata.title", "sensor_types", "event_data.host_id"]]

In [ ]:
alerts_with_asset_info = inflated_alerts.pipe(lookup_assets, env="charlie")

In [ ]:
(
    alerts_with_asset_info.drop_duplicates("asset_info.hostname")[
        [
            "metadata.title",
            "sensor_types",
            "asset_info.hostname",
        ]
    ]
)

#### Asset Data with Taegis Events

- Unlike Alert DataFrames, Taegis Event DataFrames only need the `tenant_id` and `host_id` fields

In [ ]:
%%taegis events search --assign events_df

FROM process
EARLIEST=-30d
| head 10

In [ ]:
(events_df[["enrichSummary", "host_id", "tenant_id"]])

In [ ]:
events_with_asset_info = events_df.pipe(lookup_assets, env="charlie")

In [ ]:
(events_with_asset_info[["enrichSummary", "asset_info.hostname"]])